*Copyright 2025 Jaeyoung Chun / Winning Twelve*

You may not make copies of this and use or distribute it for any purpose.

# Tokens

## Limitation and Pricing

```
| MODEL                     | CONTEXT WINDOW | MAX OUTPUT TOKENS |
|---------------------------|----------------|-------------------|
| gpt-4o-mini               | 128,000 tokens | 16,384 tokens     |
| gpt-4o-mini-2024-07-18    | 128,000 tokens | 16,384 tokens     |
| gpt-3.5-turbo-instruct    |   4,096 tokens |  4,096 tokens     |
```

```
| MODEL                     | INPUT    | CACHED INPUT | OUTPUT   |
|---------------------------|----------|--------------|----------|
| gpt-4o-mini               | $0.15    | $0.075       | $0.60    |
```

- Price per 1M tokens (as of Feb 19, 2025)
- Reference: https://platform.openai.com/docs/models/, https://platform.openai.com/docs/pricing
- Helpful Article: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
- Interactive Tokenzier: https://platform.openai.com/tokenizer

## Check Token Usage

### Chat Completions API

In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
out = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Who won the World Cup in 2022?"
        }
    ]
)

In [4]:
out.choices[0].message.content

"Argentina won the 2022 FIFA World Cup, defeating France in the final. The match, held on December 18, 2022, ended in a 3-3 draw after extra time, and Argentina triumphed 4-2 in the penalty shootout. This victory marked Argentina's third World Cup title, with previous wins in 1978 and 1986."

In [5]:
out.usage

CompletionUsage(completion_tokens=79, prompt_tokens=17, total_tokens=96, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

### Invoke API

In [6]:
from langchain_openai import ChatOpenAI

In [7]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini"
)

In [8]:
out = chat.invoke("Who won the FIFA World Cup in 2022?")

In [9]:
out.content

'Argentina won the FIFA World Cup in 2022. They defeated France in the final, which took place on December 18, 2022, in Qatar. The match ended with a score of 3-3 after extra time, and Argentina triumphed 4-2 in the penalty shootout.'

In [10]:
out.usage_metadata

{'input_tokens': 18,
 'output_tokens': 64,
 'total_tokens': 82,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

## Count Tokens

Determining the number of tokens in a text string helps assess both its compatibility with a language model’s processing limits and the associated cost of an API call.

In [11]:
import tiktoken

In [12]:
# get the tokenizer corresponding to a specific model in the OpenAI API
encoder = tiktoken.encoding_for_model("gpt-4o-mini")

### Word

### English

In [13]:
tokens = encoder.encode("internationalization")

In [14]:
tokens

[173524, 2860]

In [15]:
len(tokens)

2

In [16]:
for token in tokens:
    print(token, encoder.decode_single_token_bytes(token).decode())

173524 international
2860 ization


### Korean

있을 유(有) + 쓸 용(用)

In [17]:
tokens = encoder.encode("유용")

In [18]:
tokens

[15483, 8423]

In [19]:
len(tokens)

2

In [20]:
for token in tokens:
    print(token, encoder.decode_single_token_bytes(token).decode())

15483 유
8423 용


### Sentence

#### English

In [21]:
tokens = encoder.encode("ChatGPT is useful!")

In [22]:
tokens

[14065, 162016, 382, 8316, 0]

In [23]:
len(tokens)

5

In [24]:
for token in tokens:
    print(token, encoder.decode_single_token_bytes(token).decode())

14065 Chat
162016 GPT
382  is
8316  useful
0 !


#### Korean

In [25]:
question = "ChatGPT는 유용합니다."

In [26]:
tokens = encoder.encode(question)

In [27]:
tokens

[14065, 162016, 2770, 22748, 8423, 24490, 13]

In [28]:
len(tokens)

7

In [29]:
for token in tokens:
    print(token, encoder.decode([token]))

14065 Chat
162016 GPT
2770 는
22748  유
8423 용
24490 합니다
13 .


### Long Text

In [30]:
# https://www.newsis.com/view/NISX20250122_0003041476
text = """
스페인 프로축구 프리메라리가 바르셀로나가 벤피카(포르투갈)와 도합 9골이 터진 난타전 끝에 극적인 역전승을 거두면서 유럽축구연맹(UEFA) 챔피언스리그(UCL) 16강에 진출했다.

바르셀로나는 22일(한국 시간) 포르투갈 리스본의 이스타디우 다 루스에서 열린 벤피카와의 대회 리그 페이즈 7차전 원정 경기에서 5-4로 이겼다.

이날 승리로 바르셀로나는 승점 18(6승 1패)로 2위를 달렸고, 오는 31일 8차전 유벤투스(이탈리아)와의 맞대결 결과와 관계없이 16강행을 확정했다.

UCL 리그 페이즈는 36개 팀 중 1~8위가 16강 직행, 9~24위가 16강 플레이오프를 치른다.

바르셀로나는 리그 페이즈에서 단 1패밖에 내주지 않으며 순조롭게 16강 진출을 확정했다.

안방에서 패배한 벤피카는 승점 10(3승 1무 3패)으로 18위에 머물렀다.

벤피카는 전반전에 3-1, 후반전에 4-2로 앞서갔으나 끝내 역전을 허용하며 무릎을 꿇었다.

벤피카가 킥오프 이후 곧장 균형을 깼다.

전반 2분 반젤리스 파블리디스가 알바로 페르난데스가 건넨 크로스를 슈팅해 골망을 갈랐다.

바르셀로나는 전반 13분 로베르트 레반도프스키의 페널티킥 득점에 힘입어 반격했다.

하지만 벤피카 공격수 파블리디스가 전반 22분 추가골을 넣고, 전반 30분 해트트릭을 완성하면서 바르셀로나에 찬물을 끼얹었다.

후반전도 비슷한 양상이었다. 바르셀로나는 후반 19분 하피냐 골로 고삐를 당겼지만, 벤피카는 후반 23분 상대 수비수의 자책골로 달아났다.

바르셀로나는 포기하지 않았다. 후반 33분 레반도프스키의 페널티킥 멀티골로 3-4, 후반 41분 에릭 가르시아의 득점으로 4-4까지 쫓아갔다.

마지막 해결사는 하피냐였다.

후반 51분 전방으로 향한 롱볼이 침투하는 하피냐에게 전달됐다. 하피냐는 수비를 제친 뒤 슈팅했고 구석을 꿰뚫어 역전골을 터뜨렸다.

결국 경기는 바르셀로나의 짜릿한 5-4 승리로 막을 내렸다.
"""

In [31]:
tokens = encoder.encode(text)

In [32]:
len(tokens)

716